Install Detectron2 from GitHub

In [1]:
import sys, os, distutils.core
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-3ouko6ch
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-3ouko6ch
  Resolved https://github.com/facebookresearch/detectron2.git to commit 864913f0e57e87a75c8cc0c7d79ecbd774fc669b
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 15.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for detectron2: filename=detectron2-0.6-cp310-cp310-linux_x86_64.whl size=6119638 sha256=21a846537d0d5f84d57dee6e94e55

Import necessary libraries

In [2]:
# Import Torch framework
import torch
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# For EXIF Data Extract
import PIL
from PIL.ExifTags import TAGS
from PIL import Image
import pandas as pd

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

Parameters

In [4]:
CONFIG = "/content/drive/MyDrive/Colab Notebooks/Models/Detectron2/config.yaml"
MODEL_PATH = "/content/drive/MyDrive/Colab Notebooks/Models/Detectron2/model_final.pth"
THRESHOLD = 0.9
IMAGE_PATH = "/content/drive/MyDrive/Colab Notebooks/Data/Trash Can Data/Test/Test/Test_20.JPG"
IMAGE_FOLDER_PATH = "/content/drive/MyDrive/Colab Notebooks/Data/Trash Can Data/Test/Test"
IMAGE_OUT_PATH = "/content/drive/MyDrive/Colab Notebooks/Data/Trash Can Data/Test/Output Results"

Set up a Configuration Instance

In [5]:
def config(config_path, model_path, threshold):
  cfg = get_cfg()
  cfg.merge_from_file(config_path)
  cfg.MODEL.WEIGHTS = model_path
  cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = threshold   # set a custom testing threshold
  return cfg

Set Predictor Instance

In [6]:
def predictor(cfg):
  predictor = DefaultPredictor(cfg)
  return predictor

Function for uploading and predicting on a single image

In [9]:
def im_predict(im_path, show = True, out_folder = None):
  image = cv2.imread(im_path)
  outputs  = predictor(image)
  # Draw instance predictions on top of image
  v = Visualizer(image[:, :, ::-1])
  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  if show:
    cv2_imshow(out.get_image()[:, :, ::-1])
  if out_folder:
    filename = os.path.basename(im_path).split('/')[-1]
    outpath = os.path.join(out_folder, filename)
    cv2.imwrite(outpath, out.get_image()[:, :, ::-1])

Function for Extracting EXIF data from Apple iPhone

In [10]:
def exif_iphone(image):
  exif = {
        PIL.ExifTags.TAGS[k]: v
        for k, v in image._getexif().items()
        if k in PIL.ExifTags.TAGS
    }
  return exif

Isolate the Lat and Long from the EXIF data

In [11]:
def exif_coords(exif_data):
    exif_geo = exif_data['GPSInfo']
    lat_degrees, lat_min, lat_secs = exif_geo[2]
    latitude = lat_degrees+(lat_min/60)+(lat_secs/3600)
    if exif_geo[1] == 'S':
        latitude = latitude*-1
    long_degrees, long_min, long_secs = exif_geo[4]
    longitude = long_degrees+(long_min/60)+(long_secs/3600)
    if exif_geo[3] == 'W':
        longitude = longitude*-1

    return float(latitude), float(longitude)

Create a dictionary with keys from a list

In [12]:
def create_dict_from_list(col_list):
    my_dict = {}
    for item in col_list:
        my_dict[item]=[]
    return my_dict

Function for looping over a folder of images, predicting, and saving the outputs to another folder

In [35]:
# Set variables
image_dir = IMAGE_FOLDER_PATH
output_dir = IMAGE_OUT_PATH
col_list = ['ID','Path','Name','Make','Model','Software','DateTime','Trash Can','Latitude','Longitude']
x = 0

# Create dict to store prediction outputs
data_table = create_dict_from_list(col_list)

# Instantiate config and predictor
cfg = config(CONFIG, MODEL_PATH, THRESHOLD)
predictor = predictor(cfg)

# Loop over the images in the input folder
images = os.listdir(image_dir)
for path, image_name in enumerate(images):
  name, ext = image_name.split('.')
  if ext == 'JPG':
    image_path = os.path.join(image_dir, image_name)
    exif_image = Image.open(image_path)
    pred_image = cv2.imread(image_path)

    # Read and store EXIF data of image
    exif = exif_iphone(exif_image)
    latitude, longitude = exif_coords(exif)

    data_table['ID'].append(x)
    data_table['Path'].append(str(path))
    data_table['Name'].append(str(image_name))
    data_table['Make'].append(exif['Make'])
    data_table['Model'].append(exif['Model'])
    data_table['Software'].append(exif['Software'])
    data_table['DateTime'].append(exif['DateTime'])
    data_table['Latitude'].append(latitude)
    data_table['Longitude'].append(longitude)

    # Perform prediction on the new image
    outputs = predictor(pred_image)

    count = len(outputs["instances"])
    data_table['Trash Can'].append(count)

    # We can use `Visualizer` to draw the predictions on the image.
    v = Visualizer(pred_image[:, :, ::-1])
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

    # Create the output filename with _result extension
    result_filename = name + ".jpg"
    output_path = os.path.join(output_dir, result_filename)

    # Save the segmented image
    cv2.imwrite(output_path, out.get_image()[:, :, ::-1])

    x+=1

print("Segmentation of all images completed.")

# Convert the data table to an excel and save to the output folder
df = pd.DataFrame(data_table)
out_table = os.path.join(output_dir,"Results_Table.xlsx")
df.to_excel(out_table)

['Test_1.JPG', 'Test_2.JPG', 'Test_25.JPG', 'Test_20.JPG', 'Test_23.JPG', 'Test_16.JPG', 'Test_6.JPG', 'Test_11.JPG', 'Test_9.JPG', 'Test_18.JPG', 'Test_15.JPG', 'Test_14.JPG', 'Test_8.JPG', 'Test_17.JPG', 'Test_5.JPG', 'Test_4.JPG', 'Test_13.JPG', 'Test_24.JPG', 'Test_21.JPG', 'Test_12.JPG', 'Test_19.JPG', 'Test_22.JPG', 'Test_7.JPG', 'Test_10.JPG', 'Test_3.JPG', 'Trash_Can_Test_Labels_COCO_2023-10-12-06-52-48.json', 'Predictions_Output.xlsx']
Segmentation of all images completed.
